In [ ]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import shapely.ops
import rasterio
import numpy
import matplotlib.pyplot
import pathlib
import pdal
import json

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Setup paths

In [ ]:
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wakanae\Small_test_site')
initial_path = pathlib.Path(r'initial_data')
destination_path = pathlib.Path(r'final_data')

if not (base_path/destination_path).is_dir():
    (base_path/destination_path).mkdir()

boundary_name = pathlib.Path(r'boundary\boundary.shp')
coast_name = pathlib.Path(r'north_island\north_island.shp')
background_dem_name = pathlib.Path(r'background_dem\NZDEM_SoS_v1-0_16_Wellington_gf.tif')
background_bathymetry_name = pathlib.Path(r'background_bathymetry\sounding-points-hydro-122k-190k.shp')
lidar_name = pathlib.Path(r'points.laz')

destination_points_name = pathlib.Path(r'combined_points')
destination_dem_name_stub = pathlib.Path(r'created_dem')

# Set resolution

In [ ]:
resolution = 10

# Load in data

In [ ]:
catchment_boundary = geopandas.read_file(base_path/initial_path/boundary_name)
north_island = geopandas.read_file(base_path/initial_path/coast_name)
bathymetry = geopandas.read_file(base_path/initial_path/background_bathymetry_name)
background_dem = rioxarray.rioxarray.open_rasterio(base_path/initial_path/background_dem_name, masked=True)

# Set to same projection

In [ ]:
crs = 2193
catchment_boundary = catchment_boundary.to_crs(crs)
north_island = north_island.to_crs(crs)
bathymetry = bathymetry.to_crs(crs)
background_dem.rio.set_crs(crs);

# Get land catchment and buffered land catchment

In [ ]:
catchment_boundary_land = geopandas.clip(catchment_boundary, north_island)
bathymetry_in_catchment = geopandas.clip(bathymetry, catchment_boundary)

buffered_north_island = geopandas.GeoDataFrame(index=[0], geometry=north_island.buffer(resolution * 3), crs=crs)
coast_buffer = geopandas.clip(geopandas.overlay(buffered_north_island, north_island, how='difference'), catchment_boundary)

catchment_boundary_buffered_land = geopandas.clip(catchment_boundary, north_island.buffer(resolution * 3))

# Load in LiDAR and trim to boundary

In [ ]:
lidar_file_name = base_path/initial_path/lidar_name

pdal_pipeline_instructions = [
    {"type":  "readers.las", "filename": str(lidar_file_name)},
    {"type":"filters.reprojection","out_srs":"EPSG:" + str(crs)}, # reproject to NZTM
    {"type":"filters.crop", "polygon":str(catchment_boundary_land.loc[0].geometry)}, # filter within boundary
    {"type" : "filters.hexbin"} # create a polygon boundary of the LiDAR
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

# Trim background DEM outside LiDAR

In [ ]:
metadata=json.loads(pdal_pipeline.get_metadata())
lidar_boundary=shapely.wkt.loads(metadata['metadata']['filters.hexbin']['boundary'])

Filter areas less than some percentage

In [ ]:
percentage_to_drop = 5
area_to_drop = shapely.geometry.Polygon(lidar_boundary.exterior).area * percentage_to_drop / 100
filtered_lidar_boundary = shapely.geometry.Polygon(lidar_boundary.exterior.coords,
            [interior for interior in lidar_boundary.interiors if shapely.geometry.Polygon(interior).area > area_to_drop])
filtered_lidar_boundary=geopandas.GeoSeries([filtered_lidar_boundary])

Filter DEM inside the LiDAR region

In [ ]:
lidar_filtered_background_dem = background_dem.rio.clip([filtered_lidar_boundary.loc[0]], invert=True)

# LiDAR coast buffer

In [ ]:
pdal_pipeline_instructions = [
    {"type":  "readers.las", "filename": str(lidar_file_name)},
    {"type":"filters.reprojection","out_srs":"EPSG:" + str(crs)}, # reproject to NZTM
    {"type":"filters.crop", "polygon":str(coast_buffer.loc[0].geometry)}, # filter within coast buffer boundary
    {"type" : "filters.hexbin"}
]

pdal_pipeline_coast = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline_coast.execute();

1. Create a DEM coast buffer - where their isn't LiDAR (outside lidar coast boundary), and where in the coast buffer (inside the coast_buffer)
2. Set DEM coast buffer values - set any positive values to zero, keep negative values as negative
3. Set LiDAR coast buffer values - set any positive values to zero, keep negative values as negative

# Trim background DEM on land and coast buffer

In [ ]:
lidar_filtered_background_dem_in_catchment = lidar_filtered_background_dem.rio.clip(catchment_boundary_land.geometry)
lidar_filtered_background_dem_in_buffered_catchment = lidar_filtered_background_dem.rio.clip(catchment_boundary_buffered_land.geometry)

# Set buffered DEM coastal values to zero (or negative)
Note - there is the possibility this will miss portions of coast if the background DEM is not defined out to sea. In this case we will need to set these values to zero.

In [ ]:
lidar_filtered_background_dem_in_buffered_catchment.data[0][~numpy.isnan(lidar_filtered_background_dem_in_buffered_catchment.data[0]) 
                                                            & numpy.isnan(lidar_filtered_background_dem_in_catchment.data[0])] = 0

# Combined as a single point dataset

In [ ]:
lidar_array = pdal_pipeline.arrays[0]

dem_x, dem_y = numpy.meshgrid(lidar_filtered_background_dem_in_catchment.x, lidar_filtered_background_dem_in_catchment.y)
dem_buffered_z = lidar_filtered_background_dem_in_buffered_catchment.data[0].flatten()
dem_filtered_x = dem_x.flatten()[~numpy.isnan(dem_buffered_z)]
dem_filtered_y = dem_y.flatten()[~numpy.isnan(dem_buffered_z)]
dem_filtered_z = dem_buffered_z[~numpy.isnan(dem_buffered_z)]

bathymetry_x = bathymetry_in_catchment.apply(lambda x : x['geometry'][0].x,axis=1).to_numpy()
bathymetry_y = bathymetry_in_catchment.apply(lambda x : x['geometry'][0].y,axis=1).to_numpy()
bathymetry_z = bathymetry_in_catchment.apply(lambda x : x['geometry'][0].z,axis=1).to_numpy() * -1 # map depth to elevatation

background_x = numpy.concatenate([dem_filtered_x, bathymetry_x])
background_y = numpy.concatenate([dem_filtered_y, bathymetry_y])
background_z = numpy.concatenate([dem_filtered_z, bathymetry_z])

background_lidar = numpy.zeros_like(lidar_array, shape=[len(background_x)])
background_lidar['X'] = background_x
background_lidar['Y'] = background_y
background_lidar['Z'] = background_z

combined_points_array = numpy.concatenate([lidar_array, background_lidar])

# Save results

In [ ]:
lidar_file_name = base_path/destination_path/destination_points_name

pdal_pipeline_instructions = [
    {"type":  "writers.las", "filename": str(lidar_file_name), "a_srs": "EPSG:" + str(crs)},
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [combined_points_array])
pdal_pipeline.execute();

In [ ]:
points_arrays = pdal_pipeline.arrays

In [ ]:
dem_file_name_stub = base_path/destination_path/destination_dem_name_stub

window_sizes = range(0,11,5)
idw_power = 2
radii =  resolution * numpy.sqrt(2) * range(1,4,1)

for window_size in window_sizes:
    for radius in radii:

        pdal_pipeline_instructions = [
            {"type":  "writers.gdal", "resolution": resolution, "gdalopts":"a_srs=EPSG:" + str(crs),
             "filename": str(dem_file_name_stub) + "_window_" + str(window_size) + "_power_" + str(idw_power) + "_radius_" 
             + str(radius) + ".tiff", "output_type":["mean","idw"], "window_size": window_size, "power": idw_power, 
             "radius": radius}
        ]

        print("Window = " + str(window_size) + ", IDW Power = " + str(idw_power) + ", Radius = " + str(radius))

        pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), points_arrays)
        pdal_pipeline.execute();

# Plot the DEM

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))

catchment_boundary_buffered_land.plot(ax=ax, color='None', edgecolor='lime', linewidth=5)
filtered_lidar_boundary.plot(ax=ax, color='None', edgecolor='teal', linewidth=5)
p = ax.pcolormesh(lidar_filtered_background_dem_in_buffered_catchment.x[:], lidar_filtered_background_dem_in_buffered_catchment.y[:],
                  lidar_filtered_background_dem_in_buffered_catchment.values[0], cmap='viridis', shading='auto')
bathymetry_in_catchment.plot(ax=ax, marker='o', color='red', markersize=5)
matplotlib.pyplot.colorbar(p)

ax.set_title("Combined")